# Amazon Lookout for Vision and Amazon A2I (Augmented AI) Integration with Model Retraining

In this notebook, we will walk you through one path of how to use Amazon A2I Workteam results based from images that have returned inference results below a defined threshold from our Amazon Lookout for Vision model.

We will be using the Amazon Lookout for Vision Python SDK. It gives you a programmatic way of interacting with this service and adds a lot of helper functions that complement the service. If you have not already installed the Amazon Lookout for Vision Python SDK, we will do that in an upcoming step.

To help you learn about creating a model, Amazon Lookout for Vision provides example images of circuit boards (circuit_board) that you can use. These images are taken from https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/su-prepare-example-images.html.

In order to use this notebook successfully, make sure to clone the following repository: https://github.com/aws-samples/amazon-lookout-for-vision

The workflow for this lab is as follows:
* Define variables and install local dependencies
* Create an Amazon Lookout for Vision model using the Amazon Lookout for Vision Python SDK
* Create a human review Workteam or Workforce
* Create a Human Task UI
* Use an Amazon Lookout for Vision model to check images and then start a human loop based on inference results
* Check the status of our Human Loops, waiting for the workers to complete open tasks
* Retrain your Amazon Lookout for Vision model using the results from the human tasks

In [ ]:
# If you have not cloned the repository yet, uncomment and execute the following:

#!git clone https://github.com/aws-samples/amazon-lookout-for-vision.git

### Prerequisites and environmental variables

First, let's install required libraries and define global variables.

Libraries:
* Amazon Lookout for Vision SDK

Variables:
* region: set this to the region where your project is located
* project_name: this is the name of your Amazon Lookout for Vision project
* bucket: provide the name of the S3 bucket where we will output the model results
* a2i_output_path: provide the name of the S3 folder where A2I results will be stored
* model_version: default setting is 1
* workteam_arn: provide the name of the workforce or workteam ARN that you created

In [ ]:
# Install the Amazon Lookout for Vision SDK on the local notebook instance
# Run "pip list" to viewed all currently installed libraries if needed

#pip list

!pip install lookoutvision --upgrade
!pip install simplejson

In [ ]:
# Set the following variables to match your Amazon Lookout for Vision project

# Set the AWS region
region = 'set your region'

# Set your project name here
project_name = 'set your project name'

# Provide the name of the S3 bucket where we will output results and store images
bucket = 'set your s3 bucket name'

# This will default to a value of 1; Since we're training a new model, leave this set to a value of 1
model_version = '1'

# Leave everything else in this cell as is; nothing else to modify!

import os
import boto3
import botocore
import io
import uuid
import time
import simplejson as json
import sagemaker
import re
import pprint

from lookoutvision.metrics import Metrics
from sagemaker import get_execution_role
from lookoutvision.manifest import Manifest

# setting up the s3 folder where a2i results will be stored
a2i_results = f"s3://{bucket}/a2i-results"

# Setting Role to the default SageMaker Execution Role
role = get_execution_role()
display(role)

timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Amazon SageMaker client
sagemaker_client = boto3.client('sagemaker', region)

# Amazon Augment AI (A2I) client
a2i = boto3.client('sagemaker-a2i-runtime')

# Amazon Lookout for Vision client
L4Vclient = boto3.client("lookoutvision")

# Amazon S3 client
S3client = boto3.client('s3', region)

# Flow definition name - this value is unique per account and region. You can also provide your own value here.
flowDefinitionName = 'fd-sagemaker-object-detection-demo-' + timestamp

# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-sagemaker-object-detection-demo-' + timestamp

You will need to add the necessary Amazon Lookout for Vision permissions to your SageMaker Execution Role shown above. Simply go to IAM in the console, select "Roles", locate the name of the role as shown above, and attach the "AmazonLookoutVisionFullAccess" and "AmazonS3FullAccess" policies. In a real deployment, we would only allow the minimum permissions needed to accomplish the task.

In [ ]:
# Create the S3 bucket if doesn't exist
!aws s3 mb s3://{bucket}

IMPORTANT: If you do not add a CORS configuration to the S3 buckets that contains your input data, human review tasks for those input data objects fail. See https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html#a2i-cors-update for instructions.

To set the required CORS headers on the S3 bucket that contains your input images in the S3 console, follow the directions detailed in https://docs.aws.amazon.com/AmazonS3/latest/user-guide/add-cors-configuration.html. Use the following CORS configuration code for the buckets that host your images. If you use the Amazon S3 console to add the policy to your bucket, you must use the JSON format.

[{
   "AllowedHeaders": [],
   "AllowedMethods": ["GET"],
   "AllowedOrigins": ["*"],
   "ExposeHeaders": []
}]

### 1. Upload Circuit Board images to S3

Now it's time to upload all the images:

In [ ]:
# Upload images to S3 bucket:
# Make sure you are in the correct directory to copy the circuitboard dataset or adjust the path accordingly

!aws s3 cp ../circuitboard/train/normal s3://{bucket}/{project_name}/training/normal --recursive
!aws s3 cp ../circuitboard/train/anomaly s3://{bucket}/{project_name}/training/anomaly --recursive

!aws s3 cp ../circuitboard/test/normal s3://{bucket}/{project_name}/validation/normal --recursive
!aws s3 cp ../circuitboard/test/anomaly s3://{bucket}/{project_name}/validation/anomaly --recursive

### 2. Create an Amazon Lookout for Vision Project

You have a couple of options on how to create your Amazon Lookout project (console, CLI or boto3). We chose boto3 SDK in this example. We highly recommend to check out the console, too.

The steps we take with the SDK are:

*Create a project (the name as been defined at the beginning)
*Tell your project where to find your training dataset. This is done via the manifest file for training.
*Tell your project where to find your test dataset. This is done via the manifest file for test.
    *Note: This step is optional. In general all 'test' related code, etc. is optional. Amazon Lookout for Vision will also work with 'training' dataset only. We chose to use both as training and testing is a common (best) practice when training AI/ML models. And we should always let our customer know this to help them get to the next level.
Create a model. This command will trigger the model training and validation.

*Note: Training a model can take a few hours as it uses Deep Learning in the background. Once your model is trained, you can continue with this notebook to make predictions.

Create a model. This command will trigger the model training and validation.
Note: Training a model can (will) take a few hours as it uses Deep Learning in the background. Once your model is trained, you can continue with this notebook to make predictions.

### 2.1 Create a manifest file from the dataset

Now that we have our "circuitboard" images cloned to our local notebook at "./circuitboard", we need to generate a manifest file for training. Amazon Lookout for Vision uses this manifest file to determine the location of the files, as well as the labels associated with the files.

In [ ]:
#Create the manifest file

from lookoutvision.manifest import Manifest
mft = Manifest(
    bucket=bucket,
    s3_path="{}/".format(project_name),
    datasets=["training", "validation"])
mft_resp = mft.push_manifests()
print(mft_resp)

### 2.2 Create an Amazon Lookout for Vision Project


In [ ]:
#Create an Amazon Lookout for Vision Project

from lookoutvision.lookoutvision import LookoutForVision
l4v = LookoutForVision(project_name=project_name)
# If project does not exist: create it
p = l4v.create_project()
print(p)
print('Done!')

### 3. Create and train a model

### 3.1 Create the training and test datasets from images in S3

In [ ]:
dsets = l4v.create_datasets(mft_resp, wait=True)
print(dsets)
print('Done!')

### 3.2 Fit a model

In [ ]:
l4v.fit(
    output_bucket=bucket,
    model_prefix="mymodel_",
    wait=True)

### 3.3 Wait for the model to finish training

Training a model can (will) take a few hours as it uses Deep Learning in the background. Once your model is trained, you can continue with this notebook to make predictions. You can monitor the training progress by going to the Amazon Lookout for Vision Console and selecting the project, or by using "describe_model" below.

### 3.4 View the metrics of your model

Let's check the metrics of the model.

In [ ]:
met = Metrics(project_name=project_name)

met.describe_model(model_version=model_version)

### 3.5 Host the model

In [ ]:
l4v.deploy(
    model_version=model_version,
    wait=True)

## 4. Setup Amazon A2I (Augmented Reality)

Now that we've successfully created a dataset and trained an Amazon Lookout for Vision model, we'll setup Amazon A2I.

In this section, we will create a Workteam as well as a custom Human Task User Interface (UI)

### 4.1 Creating human review Workteam or Workforce

A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers.

When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create your Workteam, visit the instructions here: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html

After you have created your workteam, replace workteam_arn below.

In [ ]:
# Define workteam ARN

workteam_arn = 'arn:aws:sagemaker:us-east-1:714751237672:workteam/private-crowd/lfv-a2i'

### 4.2 Create Human Task UI
Create a human task UI resource, giving a UI template in liquid html. This template will be rendered to the human workers whenever human loop is required.

For over 70 pre built UIs, check: https://github.com/aws-samples/amazon-a2i-sample-task-uis.

In [ ]:
# In this section, we are creating a custom Human Task UI. When a human loop is created, workers will see this template.
# For this notebook, the worker will simply need to select if the image is "Normal" or an "Anomaly".

template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <crowd-image-classifier
    name="crowd-image-classifier"
    src="{{ task.input.taskObject | grant_read_access }}"
    header="Please select the correct category for this image"
    categories="['Normal', 'Anomaly']"
  >
    <full-instructions header="Classification Instructions">
      <p>Read the task carefully and inspect the image.</p>
      <p>Choose the appropriate label that best suits the image.</p>
    </full-instructions>

    <short-instructions>
      <p>Read the task carefully and inspect the image.</p>
      <p>Choose the appropriate label that best suits the image.</p>
    </short-instructions>
  </crowd-image-classifier>
</crowd-form>
"""

def create_task_ui():
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker_client.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
    return response

In [ ]:
# Create task UI
humanTaskUiResponse = create_task_ui()
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

### 4.3 Create the flow definition

In this section, we're going to create a flow definition definition. Flow Definitions allow us to specify:

The workforce that your tasks will be sent to.
The instructions that your workforce will receive. This is called a worker task template.
The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
Where your output data will be stored.
This demo is going to use the API, but you can optionally create this workflow definition in the console as well.

For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

In [ ]:
## create flow definition. this specifies the work team & human task UI arn

create_workflow_definition_response = sagemaker_client.create_flow_definition(
        FlowDefinitionName = flowDefinitionName,
        RoleArn = role,
        HumanLoopConfig = {
            "WorkteamArn": workteam_arn,
            "HumanTaskUiArn": humanTaskUiArn,
            "TaskCount": 1,
            "TaskDescription": "Select if the component is damaged or not.",
            "TaskTitle": "Verify if the component is damaged or not"
        },
        OutputConfig={
            "S3OutputPath" : a2i_results
        }
    )
flowDefinitionArn = create_workflow_definition_response['FlowDefinitionArn'] # let's save this ARN for future use

In [ ]:
# Describe flow definition - status should be active before proceeding

for x in range(60):
    describeFlowDefinitionResponse = sagemaker_client.describe_flow_definition(FlowDefinitionName=flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

## 4.4 Make predictions and start a human loop based on inference results

In the next section, we will loop through an array of new images, using the Amazon Lookout for Vision SDK to determine if our input images are damaged or not and if they're above or below a defined threshold; in this case, we're setting the threshold confidence at .70. If our result is below .70, we'll start a human loop for a worker to manually determine if our image is normal or an anomally.

In [ ]:
from IPython.display import Image
from IPython.display import display

# create an array of input images from local storage using the extra images from the dataset
Incoming_Images_Dir = "../circuitboard/extra_images"
Incoming_Images_Array = os.listdir(Incoming_Images_Dir)

print("Checking " + str(len(Incoming_Images_Array)) + " images. Here we go!")
print("\n")

human_loops_started = []

SCORE_THRESHOLD = .70

for fname in Incoming_Images_Array:
    #Lookout for Vision inference using detect_anomalies
    fname_full_path = (Incoming_Images_Dir + "/" + fname)
    #display fname_full_path image
    display(Image(width="400",filename=fname_full_path))
    with open(fname_full_path, "rb") as image:
        modelresponse = L4Vclient.detect_anomalies(
            ProjectName=project_name,
            ContentType="image/jpeg",  # or image/png for png format input image.
            Body=image.read(),
            ModelVersion=model_version,
            )
        modelresponseconfidence = (modelresponse["DetectAnomalyResult"]["Confidence"])
    print("Name of local file pulled from the array: " + fname)
    print("Is the PCB damaged? " + str(modelresponse["DetectAnomalyResult"]["IsAnomalous"]))
    print("Confidence: " + str(modelresponse["DetectAnomalyResult"]["Confidence"]))
    print("\n")
    #End Lookout for Vision inference
    if (modelresponseconfidence < SCORE_THRESHOLD):
        #s3_fname='s3://%s/a2i-results/%s' % (bucket, fname)
        s3_fname = (a2i_results + "/" + fname)
        #copy local image to s3 for a2i if we're below the threshold so it can be seen in the human loop
        print("Copying local image to S3 for A2I because confidence is below the threshold")

        !aws s3 cp {fname_full_path} {a2i_results}/
            
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            "initialValue": modelresponseconfidence,
            "taskObject": s3_fname
        }
        # start an a2i human review loop with an input
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=flowDefinitionArn,
            HumanLoopInput={
                "InputContent": json.dumps(inputContent)
            }
        )
        human_loops_started.append(humanLoopName)
        print("\n")
        print(f'Object detection Confidence Score of %s is less than the threshold of %.2f' % (modelresponseconfidence, SCORE_THRESHOLD))
        print(f'Starting human loop with name: {humanLoopName}  \n')
        print("-------------------------------------------------------------------------------------------")
    else:
        print(f'Object detection Confidence Score of %s is above than the threshold of %.2f' % (modelresponseconfidence, SCORE_THRESHOLD))
        print('No human loop created. \n')
        print("-------------------------------------------------------------------------------------------")

### 4.4.1 Check the status of the human loops

In [ ]:
# Check the status of the human loops

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('\n')
    
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

### 4.4.2 Wait For Workers to Complete Task
Since we are using a private workteam, we should go to the labeling UI to perform the inspection ourselves.

In [ ]:
# hint: use the output from the inference results above to help guide your answers for anomaly or normal!

workteamName = workteam_arn[workteam_arn.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker_client.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

### 4.4.3 Check the status of the human loop again

In [ ]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('\n')
    
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

### 4.4.4 View the task results and move the taskobject to the correct folder for retraining 

Once work is completed, Amazon A2I stores results in your S3 bucket and sends a Cloudwatch event. Your results should be available in the S3 a2i_output when all work is completed.

Now that we have results from our human loops stored in s3, we can use that data to sort our images into the appropriate training folders and train a new model version!

In [ ]:
# view the output of the human loop task and move the taskobject to the correct folder for retraining

pp = pprint.PrettyPrinter(indent=4)

for resp in completed_human_loops:
    split_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = split_string[1]
    response = S3client.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response["Body"].read()
    json_output = json.loads(content)
    pp.pprint(json_output)
    print('\n')
    #Let's move the taskobject image to the correct folder for retraining now
    humanAnswersResponse = (json_output["humanAnswers"])
    labelanswer = humanAnswersResponse[0]["answerContent"]["crowd-image-classifier"]["label"]
    # let's also grab the taskObject value - we'll use this to move the object to the training folder in s3
    inputContentResponse = (json_output["inputContent"])
    taskObjectResponse = (inputContentResponse["taskObject"])
    # view the results from the label
    print("The results from this human loop show the image is " + labelanswer)
    print('\n')
    # move the image to the appropriate training folder
    if (labelanswer == "Normal"):
        # move object to the Normal training folder s3://a2i-lfv-output/image_folder/normal/
        !aws s3 cp {taskObjectResponse} s3://{bucket}/{project_name}/training/normal/
        print('\n')
        print(taskObjectResponse + " has been moved to the Normal folder for training")
    else:
        # move object to the Anomaly training folder
        !aws s3 cp {taskObjectResponse} s3://{bucket}/{project_name}/training/anomaly/
        print('\n')
        print(taskObjectResponse + " has been moved to the Anomaly folder for training")
    print("------------------------------------------------------------------------------------------------------")

## 5. Training a new model from the modified dataset

Training a new model version can be triggered as a batch job on a schedule, manually as needed, based on how many new images have been added to the training folders, etc.

For this example, we will use the Lookout for Vision SDK to retrain our model using the images that we've now included in our modified dataset.

https://github.com/awslabs/amazon-lookout-for-vision-python-sdk/blob/main/example/lookout_for_vision_example.ipynb

In [ ]:
# Generate a new manifest

from lookoutvision.lookoutvision import LookoutForVision

print(f'bucket = {bucket}, project name = {project_name}')
mft = Manifest(
    bucket=bucket,
    s3_path="{}/".format(project_name),
    datasets=["training", "validation"])
print(f'mft = {mft}')
mft_resp = mft.push_manifests()
print (f'mft resp = {mft_resp}')
dsets = l4v.update_datasets(mft_resp, wait=True)

In [ ]:
# Train the model!

l4v.fit(
    output_bucket=bucket,
    model_prefix="mymodelprefix_",
    wait=True)

### View the metrics of the new model

Now that we've trained a new model using newly added images, let's check the model metrics!
We'll show the results from the first model and the second model this time.

In [ ]:
# All models of the same project
met.describe_models()

## Stop the model and cleanup resources

Be sure to stop any hosted models, delete Jupyter notebooks that are no longer used, delete any Amazon Lookout for Vision projects you are no longer using, and remove objects from S3 to save costs!

In [ ]:
#If you are not using the model, stop to save costs! This can take up to 5 minutes.

#change the model version to whichever model you're using within your current project
model_version='1'
l4v.stop_model(model_version=model_version)

# print('Stopping model version ' + model_version  + ' for project ' + project_name )
# response=L4Vclient.stop_model(ProjectName=project_name,
#     ModelVersion=model_version)
# print('Status: ' + response['Status'])